# Tutorial 4: Additional Temporal Visualizations

## **Learning Goals**
Those who actively work through this tutorial will be able to:
- Use graphical marks to create visualizations for sequential temporal tasks (e.g., line, area, and stacked area charts)
- Create visualizations for cyclic temporal tasks (e.g., heatmaps)
- Apply faceting syntax correctly using `.facet(column='field:N', columns=3)` instead of deprecated `.encode(column=...)` syntax
- Create systematic country comparisons by selecting strategic country subsets for faceted temporal analysis
- Implement scale resolution decisions using `.resolve_scale(y='independent')` when appropriate for comparative analysis
- Design slope graphs using .mark_line() with point=True to emphasize before/after change patterns
- Structure period comparison data by reshaping datasets for two-time-point analysis (e.g., 2000 vs 2020)
- Interpret change patterns by reading slope directions, magnitudes, and cross-country comparisons


## Dataset Description

We will continue to use the [**Our World in Data (OWID) Energy Dataset**](https://ourworldindata.org/electricity-mix) introduced in class.
The dataset includes metrics on energy production, consumption, and technology adoption across countries worldwide.

Here is a description of the key columns you'll be working with:

| Column                        | Description                                                            | Unit                |
|-------------------------------|------------------------------------------------------------------------|---------------------|
| `country`                     | Country name                                                           | Geographic location |
| `year`                        | Year of observation                                                    | Date                |
| `nuclear_electricity`         | Electricity generation from nuclear                                    | TWh                 |
| `hydro_electricity`           | Electricity generated from hydropower                                  | TWh                 |
| `fossil_fuel_consumption`     | Primary energy consumption from fossil fuels                           | TWh                 |
| `coal_production`             | Coal production volume                                                 | TWh                 |
| `coal_consumption`            | Coal consumption volume                                                | TWh                 |
| `biofuel_consumption`         | Primary energy consumption from biofuels                               | TWh                 |
| `gas_consumption`             | Primary energy consumption from gas                                    | TWh                 |
| `oil_consumption`             | Primary energy consumption from oil                                    | TWh                 |
| `hydro_consumption`           | Primary energy consumption from hydropower                             | TWh                 |
| `other_renewable_consumption` | Primary energy consumption from other renewables                       | TWh                 |
| `wind_electricity`            | Electricity generation from wind power                                 | TWh                 |
| `solar_electricity`           | Electricity generation from solar power                                | TWh                 |
| `hydro_electricity`           | Electricity generation from hydropower                                 | TWh                 |

**Note:** TWh = Terawatt-hours (1 TWh = 1 billion kilowatt-hours)

## Dataset and Environment Setup

In [ ]:
import pandas as pd
import altair as alt


# If on PL use this one
filepath = 'data/owid_dataset.csv',

# If running locally on your machine use this one
filepath = 'https://raw.githubusercontent.com/kemiolamudzengi/dsci-320-datasets/main/owid_dataset.csv'


# Load the OWID energy dataset and only keep from 1974 onwards 
owid_data = (pd.read_csv(filepath, parse_dates=['year'])
             .query('year.dt.year >= 1974'))
print(owid_data.shape)

<div style="border-left: 5px solid #FFD700; padding: 1em; background-color: #FFFBEA;">

<h3><b> Data Task: Preparing Energy Consumption Dataset</b></h3>

<ul>
<li>Start with the full <code>owid_data</code> dataset</li>
<li>STEP 1: Based on the four selected countries create a subset of the data (i.e choose the rows of interest) and store in <code>country_subset</code> </li>
<li>STEP 2: Based on the energy consumption create a subset from <code>country_subset</code>. It should include the year, country and the named consumption columns </li>
    <li>STEP 3: Reshape the data (store in the <code>energy_consumed</code> variable) into long format using <code>melt</code>:
<ul>
<li>Identifier variables: <code>country</code> and <code>year</code></li>
<li>Variable column: <code>energy_type</code></li>
<li>Value column: <code>consumption</code></li>
</ul>
</li>
<li>Rename energy types for clarity using the provided object <code>energy_type_names</code>

</ul>
<p> You must create all the intermediary dataframes so that you can be awarded credit for each step. </p>
</div>


In [ ]:

countries = ['Germany', 'Canada', 'Indonesia', 'Brazil']
energies = [ 'gas_consumption','oil_consumption','hydro_consumption','coal_consumption','nuclear_consumption']

# STEP 1: SELECT THE COUNTRIES OF INTEREST (select the rows of interest)
country_subset = ...

# STEP 2: SELECT the Energy Consumptions of interest (select the columns of interest)
selected = ...

# STEP 3: RESHAPE the data into long format
energy_consumed = ... # FILL IN THE BLANKS

# STEP4: CLEAN energy type names
energy_type_names = {
    'gas_consumption': 'Gas',
    'oil_consumption': 'Oil',
    'hydro_consumption': 'Hydro',
    'coal_consumption': 'Coal',
    'nuclear_consumption': 'Nuclear',
}
energy_consumed['energy_type'] = energy_consumed['energy_type'].map(energy_type_names)

energy_consumed.sample(10)


<div style="border-left: 5px solid #007BFF; padding: 1em; background-color: #F0F8FF;">

<h3><b>Viz Task: Faceted Energy Transition Area Chart</b></h3>

<ul>
<li>Use the <code>area</code> mark to show the share of different energy types over time.</li>
<li>Encode:
<ul>
<li><code>year</code> on the <b>x channel</b> as temporal.</li>
<li><code>percentage</code> on the <b>y channel</b> as quantitative. </li>
<li><code>energy_type</code> on the <b>color channel</b> </li>
</ul>
</li>
<li>Facet the chart by <code>country</code> into multiple small multiples, arranged in 3 columns.</li>
<li>Set the size to <b>200 × 200</b> for each chart.</li>
<li>Add a descriptive title: <i>“Primary energy consumption by source (Faceted by Country)”</i>.</li>
</ul>
</div>



In [ ]:

energy_small_multiples = alt.Chart(energy_consumed).mark_line().encode(
    x=alt.X('year:T', title='Year'),
    y=alt.Y('consumption:Q', title='Consumption (TWh)'),
    color=alt.Color('energy_type:N', title='Energy Type')
).properties(
    width=200, height=200
).facet(
    column=alt.Column('country:N', title=None)  # suppress auto facet header title
).properties(
    title="Primary energy consumption by source (Faceted by Country)"
)


energy_small_multiples


## Independent Y-axis

Let's see what happens when we include China in our analysis.**

<div style="border-left: 5px solid #FFD700; padding: 1em; background-color: #FFFBEA;">
    <h3>go back to the data wrangling step and add China to the list of countries </h3>
    </div>

When you add China to the energy consumption visualization, you'll immediately notice something interesting - the y-axis scale dramatically shifts upward. This happens because China's energy consumption is so much larger than most other countries that it forces the entire scale to accommodate those massive values.

**But this creates a problem.** Suddenly, the energy consumption patterns for smaller countries become nearly invisible - they all get squished down near the bottom of the chart, making it really difficult to spot trends or compare changes over time for those nations.

**So what can we do about this?  Use independent scales** 
```python
.resolve_scale(y='independent')
```
This lets each country have its own y-axis range, so you can see patterns clearly in every facet.


In [ ]:
energy_small_multiples = ...
energy_small_multiples

Every solution has its pros and cons. What do you think is the disadvantage of this new approach.

## Slope Chart

The `line` mark can also be used to create *slope graphs*, charts that highlight the change in value between two comparison points using line slopes.
By default, Altair places the years close together. To better space out the years along the x-axis, we can indicate the size (in pixels)
of discrete steps along the width of our chart as indicated by the comment below. Try adjusting the width `step` value below and see how the chart changes in response.

<div style="border-left: 5px solid #FFD700; padding: 1em; background-color: #FFFBEA;">

<h3><b>Data Task: Period Comparison of Renewable Electricity 
    (Pre- vs Post-Paris Agreement)</b></h3>
    <h4>Create a function called <code>calculate_period_averages</code> to do the following
    </h4>  
<ul>
<li>Define two comparison periods:
<ul>
<li><b>Period 1</b>: 2005–2009 (Pre-Paris Agreement)</li>
<li><b>Period 2</b>: 2015–2019 (Post-Paris Agreement)</li>
</ul>
</li>
<li>For each selected country, calculate the average <code>renewables_electricity</code> in both periods.</li>
<li>Compute change metrics:
<ul>
<li><b>Absolute Change</b>: Period 2 average – Period 1 average</li>
<li><b>Percent Change</b>: (Absolute Change ÷ Period 1 average) × 100</li>
</ul>

</div>


In [ ]:
def calculate_period_averages(data, attribute='renewables_electricity'):
    """Calculate average values for two comparison periods for all individual countries
       (excludes world/continent aggregates)."""

    results = []

    # Exclude non-country regions often found in OWID datasets
    exclude_list = [
        "World", "Africa", "Asia", "Europe", "European Union",
        "North America", "South America", "Oceania"
    ]
    
    period_1_years = list(range(2005, 2010))  # 2005-2009: Pre-Paris Agreement
    period_2_years = list(range(2015, 2020))  # 2015-2019: Post-Paris Agreement


    # Get only actual countries
    all_countries = data.loc[~data['country'].isin(exclude_list), 'country'].unique()

    for country in all_countries:
        country_data = data.query('country == @country')

        # Filter data for the two periods
        # Selects only the rows where the year is in the chosen period.
        # .dt.year extracts just the year part from a datetime column.
        
        period_1_data = country_data[country_data['year'].dt.year.isin(period_1_years)]
        period_2_data = country_data[country_data['year'].dt.year.isin(period_2_years)]

        if len(period_1_data) > 0 and len(period_2_data) > 0:
            
            # Calculate average for each time period
            period_1_avg = period_1_data[attribute].mean()
            period_2_avg = period_2_data[attribute].mean()

            # Calculate changes
            absolute_change = period_2_avg - period_1_avg
            percent_change = (absolute_change / period_1_avg * 100) if period_1_avg > 0 else 0

            # Store results
            results.append({
                'country': country,
                'period_1_avg': period_1_avg,
                'period_2_avg': period_2_avg,
                'absolute_change': absolute_change,
                'percent_change': percent_change
            })

    return pd.DataFrame(results)


<div style="border-left: 5px solid #FF1200; padding: 1em; background-color: #FF00A1; color: white;">
<h3><b>Period Comparison of Renewable Electricity (Pre- vs Post-Paris Agreement)</b></h3>
    <h4>Call the function for the <code>owid_data</code> Print out the dataframe and make sure that you understand what is going on in the function above before proceeding. 
    </h4>
</div>

Sorry that was the only way to make sure you stopped and went through the process of understanding the function above. 

In [ ]:
renewable_change = ...
renewable_change


<div style="border-left: 5px solid #FFD700; padding: 1em; background-color: #FFFBEA;">

<h3><b>Data Task: Preparing Slope Graph Data for Top Changers</b></h3>

<ul>
<li>Select the top <b>5 countries</b> with the largest <code>absolute_change</code> in renewable electricity from the period comparison.</li>
<li>Reshape the dataset into a <b>long format</b> using <code>melt</code>, keeping:
<ul>
<li><code>country</code></li>
<li><code>period</code> (before/after)</li>
<li><code>renewables_electricity</code> values</li>
</ul>
</li>
<li>Replace the generic period labels with descriptive ones:
    <ul>
    <li><code>2005–2009 (Pre-Paris)</code></li>
    <li><code>2015–2019 (Post-Paris)</code></li>
</ul>
</li>
<li>Return a tidy dataset (<code>slope_data</code>) ready for visualization in a slope graph.</li>
</ul>

</div>


In [ ]:
top_changers = renewable_change.nlargest(5, 'absolute_change')


# Reshape data for slope graph
slope_data = top_changers[['country', 'period_1_avg', 'period_2_avg']].melt(
    id_vars= ...,   # FILL IN THE BLANKS
    var_name=...,
    value_name=...,
)

# Create meaningful period labels
period_labels = {
    'period_1_avg': '2005-2009\n(Pre-Paris)',
    'period_2_avg': '2015-2019\n(Post-Paris)'
}
slope_data['period_label'] = slope_data['period'].map(period_labels)
print(slope_data.head())

<div style="border-left: 5px solid #007BFF; padding: 15px; background-color: #F0F8FF; border-radius: 8px;">

<h2>Renewable Electricity Slope Graph</h2>

<p><strong>Exploratory Question:</strong>
<em>How has renewable electricity generation changed across countries before and after the Paris Agreement?</em></p>

<p><strong>VIZ TASK:</strong> Create a slope graph comparing renewable electricity growth for multiple countries between two periods.</p>

<h3>Chart Specification:</h3>
<ul>
<li>Create a base chart using the <code>slope_data</code> dataset.</li>
<li>Use <code>mark_line</code> to show changes for each country over time.</li>
<li>Encode <code>period_label</code> on the <strong>x channel</strong> as ordinal.</li>
<li>Encode <code>renewables_electricity</code> on the <strong>y channel</strong> as quantitative.</li>
<li>Encode <code>country</code> on the <strong>color channel</strong>.</li>
<li>Include tooltips for <code>country</code>, <code>period_label</code>, and <code>renewables_electricity</code>.</li>
</ul>

<h3>Styling Specifications:</h3>
<ul>
<li>Line width: <code>strokeWidth=3</code>.</li>
<li>Point size: 100 pixels.</li>
<li>Font size for labels: 11px, bold, aligned left with dx=8 offset.</li>
<li>Color scheme: <code>category10</code>.</li>
</ul>

</div>


In [ ]:
base = alt.Chart(slope_data)

# Lines showing change (core of slope graph)
lines = base.mark_line(strokeWidth=3).encode(
    x=alt.X('period_label:O', title='Time Period',
            axis=alt.Axis(labelAngle=0, labelFontSize=12)),
    y=alt.Y('renewables_electricity:Q', title='Renewable Electricity (TWh)',
            axis=alt.Axis(format='.0f')),
    color=alt.Color('country:N', title='Country', scale=alt.Scale(scheme='category10'), legend = None),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('period_label:O', title='Period'),
        alt.Tooltip('renewables_electricity:Q', title='Renewable Electricity (TWh)', format='.1f')
    ]
)

# Points at each period for precision
points = base.mark_circle(size=100, opacity = 1).encode(
    x='period_label:O',
    y='renewables_electricity:Q',
    color=alt.Color('country:N', scale=alt.Scale(scheme='category10'), legend= None),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('period_label:O', title='Period'),
        alt.Tooltip('renewables_electricity:Q', title='Renewable Electricity (TWh)', format='.1f')
    ]
)




Yes it looks like a mess, don't worry we will fix it very soon
Run the code below

In [ ]:
# Country labels at end points for identification
labels = base.mark_text(
    align='left', dx=8, fontSize=11, fontWeight='bold'
).encode(
    x='period_label:O',
    y='renewables_electricity:Q',
    text=alt.condition(
        alt.datum.period_label == '2015-2019\\n(Post-Paris)',  # compare to the Post-Paris panel
        'country:N',
        alt.value('')
    ),
    color=alt.Color('country:N', scale=alt.Scale(scheme='category10')),
)

labels

* `mark_text` → creates text marks instead of lines or points.
* `align='left', dx=8` → pushes labels slightly to the right of the data point, so they don’t overlap the line.
* `fontSize` and `fontWeight` → make the labels readable and bold.
* `x='period_label:O'` → plots text at the categorical x-axis positions (`2005–2009` vs `2015–2019`).
* `y='renewables_electricity:Q'` → places the text at the same height as the country’s renewable electricity value.
* `text=alt.condition(...)` → **only shows labels for the “Post-Paris” period**, leaving the left-hand side blank. This avoids clutter.
* `color='country:N'` → makes label colors match the country line colors.

Effect: Each line ends with a country name, so viewers can identify them without a legend.


Now we are going to combine them all together and do some styling of the combined chart.

`.resolve_scale(color='independent')` → ensures each layer (lines, points, labels) shares the same consistent color mapping.

In [ ]:

# Combine all layers and attach the param ONCE
slope_graph = alt.layer(lines, points, labels).properties(
    width=400,
    height=500,
    title=["Renewable Electricity Growth:", "Pre-Paris vs Post-Paris Agreement"]
).resolve_scale(
    color='independent'
)
slope_graph

Take a minute to let that all sink it. 
It is alot. 
I suggest you to through each line of code and write a comment explaining what each line does. 

## Rect Marks
The last mark that we will use is the `rect_mark`.
It is typically used to create heatmaps.
The term heatmap was assigned to this visual representation in the early 1990s and was widely used in the financial industry to depict cyclic time-varying data.
A heatmap is bascially a matrix or table in which each cell uses color to encodea a numerical value.
**Heatmaps** use the `rect` mark in Altair to create matrix-like visualizations where each rectangle represents a combination of two dimensions (e.g., year and country). The color of each rectangle encodes a quantitative value, making it easy to compare values across both dimensions at a glance. Heatmaps are especially useful for identifying patterns, trends, and outliers in large datasets, such as detecting which countries generate the most renewable electricity over time or highlighting changes across decades.

For more details on the capabilities and customization options of rectangle marks in Altair, see the official documentation: [Altair Rect Mark](https://altair-viz.github.io/user_guide/marks.html#rect-mark).


### Energy Data
Our owid_data is a bit boring for this (only years), so we will be visualizing a subset of Mike Bostock's energy consumption data for 2019.
To get a sense of the data, please skim the [visualization](https://observablehq.com/@mbostock/electric-usage-2019) he created.

In [ ]:

path = 'https://raw.githubusercontent.com/kemiolamudzengi/dsci-320-datasets/main/energy_usage.csv'

data = pd.read_csv(path)
data.head(10)

### TimeUnit Tranforms

Here are excerpts from the API about [Times and Dates](https://altair-viz.github.io/user_guide/times_and_dates.html?highlight=time)

> Altair is designed to work best with Pandas timeseries. A standard timezone-agnostic date/time column in a Pandas dataframe will be both interpreted and displayed as local user time.
For date-time inputs like these, it can sometimes be useful to extract particular time units (e.g. hours of the day, dates of the month, etc.).
In Altair, this can be done with a time unit transform, discussed in detail in [TimeUnit Transform](https://altair-viz.github.io/user_guide/transform/timeunit.html#user-guide-timeunit-transform).

We will provide some examples, but strongly recommend that you consult the API.

For example, we might decide we want a heatmap with hour of the day on the x-axis, and day of the month on the y-axis:
Let's start off my encoding the month for each data item with the `x` channel.

In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.X('month(date)').title('Month')
)

It is very hard to see each individual rectangle. All we can surmise from this visual representation is that the dataset includes energy data for the first 7 months of the year.
Let's use color to encode the energy usage for each month.

In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.X('month(date)').title('Month'),
    alt.Color('sum(usage):Q').title('Sum of Usage (kW)')
)

Now we can see individual rectangles. Because Mike has solar panels the energy consumption reduces as we proceed through the year.
In this visualize we are addressing a sequential task. Let's transition to time-varying tasks.

### Heatmap
Let us answer the question _What does the energy consumption look like for each day of the week?_
To create a heatmap, let's use the `y` channel to encode the day of the week.

In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.Y('day(date):O').title('Day'),
    alt.X('month(date)').title('Month'),
    alt.Color('sum(usage):Q').title('Sum of Usage (kW)')
)

Notice how for both the `x` and `y` channels we are using the same attribute/field in our dataset **date**.
The TimeUnit transform extracts the relevant aspects from the datum.
What if we wanted to ask the question, _what time of the day has the highest or lowest energy usage?_
Let's use `y` to encode the month, and `x` to encode the time of the day.


In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.Y('month(date):O').title('Month'),
    alt.X('hoursminutes(date):O').title('Hours and Minutes'),
    alt.Color('sum(usage):Q').title('Sum of usage (kW)')
)

It is worth mentioning that the color channel is encoding an aggregation.
It is not encoding the individual energy for a specific day.
Change the aggregation from `sum` to `average` what differences do you observe.
Remove the aggregation, what is being depicted?
We can ask the question again _what time of day has the highest or lowest energy usage?_ but this time let us aggregated the day by day of the week as opposed to month.

In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.Y('day(date):O').title('Day'),
    alt.X('hoursminutes(date):O').title('Hours n Minutes'),
    alt.Color('sum(usage):Q').title('Sum of usage (kW)')
)

Let's visualize the entire dataset,
Use the `y` channel to encode the date, the `x` channel to encode the time of day.


In [ ]:
alt.Chart(data).mark_rect().encode(
    alt.Y('monthdate(date):O'),
    alt.X('hoursminutes(date):O'),
    alt.Color('usage:Q')
)

Do you notice the date that has no data? Go to Mike's post](https://observablehq.com/@mbostock/electric-usage-2019) to find out why.
This is a big plot, let's switch the data being encoded on the `x` and `y` channels and make the chart smaller.
Let's rename the axes titles as well and add a title for the chart.

In [ ]:
alt.Chart(data).mark_rect().encode(
    x = alt.X('monthdate(date):O',title='Day'),
    y = alt.Y('hoursminutes(date):O',title='Time of Day'),
    color = alt.Color('usage:Q', title='Usage (kW)'),
    tooltip =[
        alt.Tooltip('monthdate(date):O', title='Day'),
        alt.Tooltip('hoursminutes(date):O', title='Time'),
        alt.Tooltip('usage:Q', title='Usage (kW)')
    ]
).properties(width=700, height = 150, title = "Mike Bostocks' Household Energy Usage 2019")

What do you observe?
Note that you can customize each rectangle and set its size. If you go that route, you have to play around with resizing the chart to make sure that there is no blank space.


<div class="alert alert-success" role="alert" style="background-color:#d4edda; color:#155724; border:1px solid #c3e6cb; padding:1rem; border-radius:0.5rem;">
  <p><strong>Success!</strong> In this tutorial, you learned how to create small multiples of line charts. The same structure can be applied to other charts.</p>
  
  <p>We also created a function that generated a new data frame, which we then used to build a <strong>slope graph</strong>. This might be the most complex visualization you’ve seen so far—it had three layers:</p>
  <ul>
    <li>Line</li>
    <li>Points</li>
    <li>Text</li>
  </ul>
  
  <p>Go back and review how each layer was created before being combined into the final chart.</p>
  
  <h4>Next Steps</h4>
  <p>Step away from our datasets and explore examples of line charts, area charts, slope graphs, and heatmaps in the Altair gallery. Pay attention to how different specifications are made. Here are a few interesting ones:</p>
  
  <ul>
    <li><a href="https://altair-viz.github.io/gallery/line_chart_with_datum.html" target="_blank">Line chart with datum</a></li>
    <li><a href="https://altair-viz.github.io/gallery/layer_line_color_rule.html" target="_blank">Layer line with color rule</a></li>
    <li><a href="https://altair-viz.github.io/gallery/line_custom_order.html" target="_blank">Line chart with custom order</a></li>
    <li><a href="https://altair-viz.github.io/gallery/window_rank.html" target="_blank">Window rank</a></li>
    <li><a href="https://altair-viz.github.io/gallery/area_faceted.html" target="_blank">Faceted area chart</a></li>
    <li><a href="https://altair-viz.github.io/gallery/annual_weather_heatmap.html" target="_blank">Annual weather heatmap</a></li>
  </ul>
</div>
